# Replicating Corporate Bond Columns from “Evidence from Many Asset Classes”

This notebook will give a brief tour of our cleaned data and some of the analysis performed in the code. 

We first load the required packages needed to load the data: 

In [20]:
import os
from pathlib import Path
from datetime import datetime

# Derive current working directory
current_dir = Path(os.getcwd())

# If current working directory is not 'src', update it
if current_dir.stem != 'src':
    src_directory = current_dir / 'src'
    os.chdir(src_directory)
    
import config

OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)


import pandas as pd
import numpy as np
import os
import re
from pathlib import Path
from datetime import datetime

We automate collection of TRACE data by connecting to the WRDS database and then export the data as a .csv file. Lehman Brothers dataset is downloaded manually from the following link:
http://www.columbia.edu/acis/eds/holdings/1021/

Mergent FISD/NAIC is also downloaded manually from WRDS as the wrds package in Python does not have full access to the variables in the table "naic_bond_transactions" of "fisd" libarary:
https://wrds-www.wharton.upenn.edu/data-dictionary/fisd_naic/naic_bond_transactions/

In [21]:
import wrds
WRDS_USERNAME = config.WRDS_USERNAME

# Connect to WRDS
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Collect TRACE data
sql_query_T = """select date,cusip,price_l5m,coupon,yield,maturity
                        from wrdsapps.bondret 
                        """
df_T = db.raw_sql(sql_query_T)

# Export output
df_T.to_csv(DATA_DIR / "TRACE.csv", index = False)

Loading library list...
Done


We then load the Lehman Brothers dataset along with the Mergent FISD/NAIC dataset. Regular expression patterns are used to read the Lehman datasets into columns.

In [22]:
import re

folder_path = DATA_DIR / 'manual/Lehman data'

In [24]:
# Define regular expression pattern, only match needed columns
pattern = re.compile(
    r'(\S{8})\s+'          # cusip
    r'.*?\s{2,}'           # skip name column
    r'(\d{8})\s+'          # date
    r'.*?\s+'              # skip idate 
    r'(\d{8})\s+'          # mdate
    r'.*?\s+'              # skip tdrmtx column
    r'(-?\d+\.\d{3})\s+'   # fprc
    r'.*?\s+'              # skip aint column
    r'(-?\d+\.\d{4})\s+'   # cp
    r'(-?\d+\.\d{3})\s+'   # yld
)

We then move to processing the dataframe to get the information we need. This is still part of the same function.

In [25]:
files = os.listdir(folder_path)
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            match = pattern.match(line)
            if match:
                # extract interested columns only
                data.append(match.groups())
                
    # specify columns
    columns = ['cusip', 'date', 'maturity', 'fprc', 'cp', 'yld']
    df = pd.DataFrame(data, columns=columns)
    dfs.append(df)

# concatenate all dataframes into a large dataframe
dfL = pd.concat(dfs, axis=0, ignore_index=True)

# convert date format
dfL['date'] = pd.to_datetime(dfL['date'], format='%Y%m%d', errors='coerce')
dfL['maturity'] = pd.to_datetime(dfL['maturity'], format='%Y%m%d', errors='coerce')
dfL = dfL.dropna(subset=['maturity'])

# convert numbers to numeric format
convert_float = ['fprc', 'cp', 'yld']
dfL[convert_float] = dfL[convert_float].apply(pd.to_numeric, errors='coerce')

# Calculate month_to_maturity 
dfL['month_to_maturity'] = (dfL['maturity'].dt.to_period('M') - dfL['date'].dt.to_period('M')).apply(lambda x: x.n)

dfL = dfL[dfL['month_to_maturity'] <= 360]


stdL = ['id', 'date', 'maturity', 'price', 'coupon', 'yield', 'month_to_maturity']
dfL = dfL.rename(columns=dict(zip(dfL.columns, stdL)))

After loading the Lehman Brothers data, we load the TRACE dataset we initially collected

In [26]:
# 2) TRACE
file_path_T = DATA_DIR / 'TRACE.csv'
dfT = pd.read_csv(file_path_T)
dfT['yield'] = dfT['yield']*100 # data automatically collected

stdT = ['date', 'id', 'price', 'coupon', 'yield', 'maturity']
dfT = dfT.rename(columns=dict(zip(dfT.columns, stdT)))
dfT['date'] = pd.to_datetime(dfT['date'], format='%Y-%m-%d')
dfT['maturity'] = pd.to_datetime(dfT['maturity'], format='%Y-%m-%d')

# dfT['yield'] = dfT['yield'].str.rstrip('%')
convert_float = ['yield', 'coupon', 'price']
dfT[convert_float] = dfT[convert_float].apply(pd.to_numeric, errors='coerce')

# Calculate month_to_maturity
dfT['month_to_maturity'] = (dfT['maturity'].dt.to_period('M') - dfT['date'].dt.to_period('M')).apply(lambda x: x.n)
dfT = dfT[dfT['month_to_maturity'] <= 360]

We then load the Mergent dataset, change the datatypes of specific columns, and only keep the rows where the day is the last day in each month, as specified in the paper. We then calculate month_to_maturity. 

In [27]:
# 3) Mergent
file_path_M = DATA_DIR / 'manual' / 'Mergent_part.csv'
dfM = pd.read_csv(file_path_M)

# Rename columns
stdM = ['id', 'price', 'coupon', 'date', 'maturity', 'yield']
dfM = dfM.rename(columns=dict(zip(dfM.columns, stdM)))


# Change data types
convert_float = ['coupon', 'price', 'yield']
dfM[convert_float] = dfM[convert_float].apply(pd.to_numeric, errors='coerce')

# Deal with "date"
dfM['date'] = pd.to_datetime(dfM['date'], format='%Y-%m-%d', errors = 'coerce')
dfM['maturity'] = pd.to_datetime(dfM['maturity'], format='%Y-%m-%d', errors = 'coerce')

# Filter useful rows in Mergent dataset
# start_date = pd.Timestamp('1998-04-01')
# end_date = pd.Timestamp('2002-06-30')
# dfM = dfM[(dfM['date'] >= start_date) & (dfM['date'] <= end_date)]

# Only keep rows where the day is the latest in each month
dfM = dfM.groupby([dfM['id'], dfM['date'].dt.year, dfM['date'].dt.month]).apply(lambda x: x.loc[x['date'].idxmax()])
dfM = dfM.reset_index(drop=True)
dfM = dfM.dropna(subset=['maturity'])

# Calculate month_to_maturity
dfM['month_to_maturity'] = (dfM['maturity'].dt.to_period('M') - dfM['date'].dt.to_period('M')).apply(lambda x: x.n)
dfM = dfM[dfM['month_to_maturity'] <= 360]
# dfM['maturity'].isna().sum()

We then merge the three datasets together.

In [28]:
# Merge 1) & 2) & 3)
df_merge = pd.concat([dfL, dfT, dfM], axis=0)

After merging the three datasets into one, we clean the merged data. The cleaning process follows Nozawa (2017). First, corporate bond prices that are lower than one cent is removed. Second, adjacent prices whose product is less than -0.04 are also removed because this may indicate there is a recording error.

In [29]:
# 1. Drop corporate price below on cent per dollar
df_drop = df_merge[~(df_merge['price'] < 0.01)]

# 2. Remove rows of adjacent returns whose product is less than -0.04
# Calculate return
df_sorted = df_drop.sort_values('date', ascending=True).reset_index(drop=True)
df_sorted['date'].is_monotonic_increasing
grouped = df_sorted.groupby('id')
df_sorted['return'] = grouped.apply(lambda x:(x['price'] + x['coupon']) / x['price'].shift(1)).reset_index(level=0, drop=True)

# Remove rows of adjacent returns whose product is less than -0.04
df_b = df_sorted.sort_values(['id', 'date'])
indices_to_remove = []

# The following code is commented out because previous running result shows that there is no such case
# You can check by bringing these code back

# for _, group in df_b.groupby('id'):
#     product = group['return'].shift(1) * group['return']
#     mask = product < -0.04
#     indices_to_remove.extend(group.loc[mask].index)

df_remove = df_b.drop(indices_to_remove)
df_b = df_remove.reset_index(drop=True)

Then, we construct interpolated risk-free rate based on constant-maturity Treasury yields to merge into our data. First, NaN values are filled using linear interpolation method, then we derive interpolated risk-free rates for maturities every month during 1 month to 360 months using the cubic splines method. We export this as a .csv file to then merge into our data.

In [30]:
from scipy.interpolate import CubicSpline

file_path = DATA_DIR / 'manual' /'Monthly Treasury Yield.csv'

# Read monthly T-bill interest rates
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Date'])

convert_numeric = ['M01', 'M03', 'M06', 'Y01', 'Y02', 'Y03', 'Y05', 'Y07', 'Y10', 'Y20', 'Y30']
for column in convert_numeric:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Interpolate df to get rf in each month
df[convert_numeric] = df[convert_numeric].interpolate(method='linear', axis=1, limit_direction='both')


# Initialize an empty list to store interpolated results
interpolated_results = []

periods = ['M01', 'M03', 'M06', 'Y01', 'Y02', 'Y03', 'Y05', 'Y07', 'Y10', 'Y20', 'Y30']  # maturities

# Define numeric representations for maturities (in year)
maturity_numeric = np.array([1/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])

# Loop through every row to interpolate yield rate
for index, row in df.iterrows():
    
    # Extract every row and corresponding columns
    monthly_data = df.loc[index, periods].values
    
    # Create cubic splines interpolation function
    cs = CubicSpline(maturity_numeric, monthly_data)

    # Create total number of maturities after interpolation
    maturity_interpolate = np.linspace(maturity_numeric.min(), maturity_numeric.max(), 360)

    # Derive the interpolation results
    interpolated_rates = cs(maturity_interpolate)
    
    interpolated_results.append(interpolated_rates)

# Turn into a dataframe
interpolated_results = pd.DataFrame(interpolated_results)

# Rename columns
new_columns = [f'M{i+1}' for i in range(360)]
interpolated_results.columns = new_columns

# Create a series of dates corresponding to the results
date_range = pd.date_range(start='1953/4/1', end='2024/1/1', freq='MS')
date_df = pd.DataFrame(date_range[:len(interpolated_results)], columns=['Date'])

# Concatenate the dates with 'zero_rate'
interpolated_results = pd.DataFrame(pd.concat([date_df, interpolated_results], axis=1))

interpolated_results.to_csv(OUTPUT_DIR / "Interpolated_Rf.csv", index=False)

We then load the interpolated risk-free rate, change the data to long format, and calculate the yield spread

In [31]:
df_b['year_month'] = df_b['date'].dt.to_period('M')

# Load interpolated rf rate
rf_rates_df = pd.read_csv(OUTPUT_DIR / 'Interpolated_Rf.csv')
rf_rates_df['Date'] = pd.to_datetime(rf_rates_df['Date'])
rf_rates_df['year_month'] = rf_rates_df['Date'].dt.to_period('M')

# Change rf to long format
rf_long_df = rf_rates_df.melt(id_vars=['Date', 'year_month'], var_name='month', value_name='rf_rate')
rf_long_df['month_to_maturity'] = rf_long_df['month'].str.replace('M', '').astype(int)

# Merge df_b and rf based on year_month and month_to_maturity
merged_df = df_b.merge(rf_long_df, on=['year_month', 'month_to_maturity'], how='left')

merged_df['excess_return'] = np.log(merged_df['return']) - np.log(merged_df['rf_rate']/100+1)
merged_df = merged_df.dropna(subset=['excess_return'])

# Calculate yield spread
merged_df['yield_spread'] = merged_df['yield'] - merged_df['rf_rate']

df_minus = merged_df

After merging these datasets together, we then replicate the corporate bond columns. We sort portfolios by yield spread, then calculate the average value of excess return for each group to derive the results

In [33]:
end_date = datetime(2023, 12, 31)
df_sum = df_minus[df_minus['date']<=end_date]

df_sum = df_sum.dropna(subset=['yield_spread'])
df_sum['date'] = df_sum['date'].dt.to_period("M")

# Sort portfolios by yield_spread
df_sum['group'] = df_sum.groupby('date')['yield_spread'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# Calculate average value of excess_return for each group
grouped = df_sum.groupby(['date', 'group'])['excess_return'].mean().reset_index()

# Derive result
result = grouped.pivot(index='date', columns='group', values='excess_return')

result = result.reset_index()

# Rename the columns
rename = result.columns[1:]
new_column_names = ['US_bonds_{:02d}'.format(i+11) for i in range(len(rename))]
columns_mapping = dict(zip(rename, new_column_names))
result.rename(columns=columns_mapping, inplace=True)

result.to_csv(OUTPUT_DIR / 'Corporate Bond Return Replicated.csv', index=False)  # export output to specified path